# Router LLM Model Agent 튜토리얼

Router LLM Model Agent는 질문의 복잡도나 특성에 따라 적절한 LLM 모델을 자동으로 선택하는 시스템이다. 간단한 질문에는 경량 모델을, 복잡한 질문에는 고성능 모델을 사용하여 비용과 성능을 최적화한다.

기본 작동 원리는 다음과 같다:
1. 사용자 질문의 복잡도를 분석한다
2. 복잡도에 따라 적절한 모델을 선택한다 (gpt-4o-mini, gpt-4o, gpt-4-turbo 등)
3. 선택된 모델로 질문에 답변한다
4. 사용된 모델 정보와 함께 결과를 반환한다

이 방식은 리소스를 효율적으로 사용하면서도 필요한 경우 강력한 모델의 성능을 활용할 수 있다.

In [63]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수를 로드한다
load_dotenv()

True

In [65]:
from openai import OpenAI
import json

# OpenAI 클라이언트 초기화
client = OpenAI()

In [66]:
# 사용 가능한 모델 목록과 특성
model_config = {
    "gpt-5-nano": {
        "name": "gpt-5-nano",
        "description": "경량 모델 - 간단한 질문에 적합",
        "complexity": "low"
    },
    "gpt-5-mini": {
        "name": "gpt-5-mini",
        "description": "중급 모델 - 일반적인 질문에 적합",
        "complexity": "medium"
    },
    "gpt-4o-mini": {
        "name": "gpt-4o-mini",
        "description": "고급 모델 - 복잡한 질문에 적합",
        "complexity": "high"
    }
}

In [67]:
def analyze_complexity(query):
    """질문의 복잡도를 분석한다"""
    
    prompt = f"""
    다음 질문의 복잡도를 분석하라.
    
    질문: {query}
    
    복잡도 기준:
    - low: 단순 사실 확인, 간단한 계산, 일반 상식
    - medium: 설명이 필요한 개념, 비교 분석, 중간 수준의 추론
    - high: 복잡한 추론, 창의적 작성, 다단계 분석, 전문 지식
    
    복잡도만 답변하라 (low, medium, high 중 하나).
    """
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
    )
    
    complexity = response.choices[0].message.content.strip().lower()
    return complexity

In [68]:
def select_model(complexity):
    """복잡도에 따라 적절한 모델을 선택한다"""
    
    # 복잡도에 따른 모델 매핑
    complexity_to_model = {
        "low": "gpt-5-nano",
        "medium": "gpt-5-mini",
        "high": "gpt-4o-mini"
    }
    
    selected_model = complexity_to_model.get(complexity, "gpt-4o-mini")
    return selected_model

In [69]:
def generate_response(query, model_name):
    """선택된 모델로 질문에 답변한다"""
    
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": query}],
    )
    
    answer = response.choices[0].message.content
    return answer

In [70]:
def router_llm_agent(query):
    """Router LLM Model Agent의 전체 파이프라인을 실행한다"""
    
    print(f"질문: {query}\n")
    
    # 1단계: 복잡도 분석
    complexity = analyze_complexity(query)
    print(f"분석된 복잡도: {complexity}\n")
    
    # 2단계: 모델 선택
    selected_model = select_model(complexity)
    model_info = model_config[selected_model]
    print(f"선택된 모델: {model_info['name']}")
    print(f"모델 설명: {model_info['description']}\n")
    
    # 3단계: 답변 생성
    answer = generate_response(query, selected_model)
    print(f"답변:\n{answer}\n")
    
    # 결과 반환
    result = {
        "query": query,
        "complexity": complexity,
        "model_used": selected_model,
        "answer": answer
    }
    
    return result

In [71]:
# 테스트 실행
test_queries = [
    "대한민국의 수도는?",
    "양자역학과 상대성이론의 차이를 설명해줘",
    "기후 변화가 경제에 미치는 영향을 다각도로 분석하고 해결책을 제시해줘"
]

results = []

for query in test_queries:
    print("=" * 80)
    result = router_llm_agent(query)
    results.append(result)
    print("=" * 80)
    print()

질문: 대한민국의 수도는?

분석된 복잡도: low

선택된 모델: gpt-5-nano
모델 설명: 경량 모델 - 간단한 질문에 적합

답변:
서울입니다.


질문: 양자역학과 상대성이론의 차이를 설명해줘

분석된 복잡도: medium

선택된 모델: gpt-5-mini
모델 설명: 중급 모델 - 일반적인 질문에 적합

답변:
좋습니다. 양자역학(양자 이론)과 상대성이론은 현대 물리학의 두 축이지만 출발점, 적용 범위, 기본 원리 등에서 크게 다릅니다. 핵심 차이를 이해하기 쉽게 정리하겠습니다.

1) 기본 목적과 적용 범위
- 양자역학: 원자·분자·전자 등 아주 작은 미시 세계(핵·입자 수준)에서 물질과 에너지의 거동을 설명합니다. 입자의 파동성, 에너지의 양자화, 확률적 결과 등을 다룹니다.
- 상대성이론: 공간과 시간, 중력의 작용을 설명합니다. 특수상대성이론(특수 상대성)은 빛의 속도 불변성과 관성계 사이의 변환을 다루며 고속(빛에 가까운 속도)에서 중요합니다. 일반상대성이론(일반 상대성)은 중력(시공간의 곡률)을 설명해 천체·우주 규모에서 적용됩니다.

2) 기본 원리와 개념적 차이
- 양자역학
  - 파동함수(ψ)와 확률해석: 입자의 위치·운동량 등은 확률분포로 기술됩니다.
  - 중첩(superposition)과 간섭: 한 입자가 여러 상태를 동시에 가질 수 있고 간섭 현상이 나타납니다.
  - 불확정성 원리(Heisenberg): 위치와 운동량처럼 특정 쌍의 물리량은 동시에 정확히 측정할 수 없습니다.
  - 관측(측정) 문제: 측정 과정에서 파동함수가 붕괴되는 문제(해석 논쟁이 있음).
- 상대성이론
  - 특수상대성: 시간과 공간이 관찰자에 따라 섞이며(시간지연, 길이수축), 빛의 속도는 모든 관성계에서 일정합니다. 질량-에너지 등가 E=mc^2.
  - 일반상대성: 질량·에너지가 시공간을 굽히고, 그 곡률이 물질의 운동을 결정합니다(중력은 힘이 아니라 시공간의 기하학적 효과).
  - 절대적 시간 없음: 시간은 관찰자에 따라 다르게 흐름(동

In [72]:
# 결과 요약
print("\n===== 전체 결과 요약 =====")
for i, result in enumerate(results, 1):
    print(f"\n{i}. 질문: {result['query']}")
    print(f"   복잡도: {result['complexity']}")
    print(f"   사용 모델: {result['model_used']}")


===== 전체 결과 요약 =====

1. 질문: 대한민국의 수도는?
   복잡도: low
   사용 모델: gpt-5-nano

2. 질문: 양자역학과 상대성이론의 차이를 설명해줘
   복잡도: medium
   사용 모델: gpt-5-mini

3. 질문: 기후 변화가 경제에 미치는 영향을 다각도로 분석하고 해결책을 제시해줘
   복잡도: high
   사용 모델: gpt-4o-mini
